# MIE1512H Project

### Topic: Predicting Station-Level Demand in Bike-Sharing Systems

Name: Tanya Tang<br>
Version : V1<br>
Date: March 11, 2020

___
### Project Overview

The goal of the paper this project is based on is to improve the effectiveness of station-level rebalancing activites within bike-sharing systems by obtaining more accurate minimum inventory levels per station. 

The authors used historical bike trip data along with hourly weather data to train several machine learning models. The model features fell within two categories, time-related features from the bike data (i.e. year, month, etc.) and weather-related features (i.e. precipitation, temperature, etc.). To reduce overfitting, the data is first transformed onto a reduced space using four different reduction techniques:
1. No reduction, baseline
2. Group all stations into one cluster
3. Kmeans - cluster stations into k clusters
4. Singular value decomposition

After reducing the problem, four different prediction methods were tested:
1. Linear regression
2. Multi-layer perceptron
3. Gradient boosted tree
4. Random forest

Once a prediction has been made on the reduced problem, the reduction needs to be inverted to obtain the full problem predictions. Each reduction technique has its own specific inversion process. 

The solution techniques were tested and scored based on the performance of its generated minimum station inventory levels. 

In this project, a similar methodology will be followed, with modifications and additions on the feature construction aspect. Focusing on the city of San Francisco, historical bike data and weather data will be used in conjunction with crime data. Features from all three data sets will be constructed, and several prediction techniques will be tested. 

___
### Project Versions/Timelines

#### V1

CRISP-DM Tasks:
* Data Understanding (collecting, describing, exploring, verifying)
* Data Preparation on Test Scale (selecting, cleaning, constructing, integrating, formatting)

Timeline:
* Looking up potential datasets and collecting from online sources/deciding which datasets to use (0.5 hr)
* Exploring entries/schema of each chosen dataset to understand discrepancies and connections between datasets (1 hr)
* Loading/cleaning raw data within each dataset, taking a closer look at which entries are relevant (2 hr)
* Calculating connections between different datasets and joining tables to eventually form a single aggregated table containing all relevant information for a single entry (4 hrs)
* Verifying data preparation steps (i.e. tables joined correctly, entries were not lost throughout the preparation process, no null values exist) (1 hr)

#### V2

CRISP-DM Tasks:
* Data Preparation on Full Scale (selecting, cleaning, constructing, integrating, formatting)
* Modeling (selecting techniques, generating test design, building, assessing)

Planned Timeline (Weeks 1 and 2: March 12 to 26):
* Recreate data preparation tasks from V1 on full dataset (1 hr)
* Research reduction/prediction techniques and how to apply them using python packages (1 hr)
* Implement reduction techniques (4 hr)
* Implement regression techniques (4 hr)
* Implement inversion techniques (4 hr)
* Test implemented models (1 hr)

#### V3

CRISP-DM Tasks:
* Evaluation (evaluting results, reviewing process, determining next steps)

Planned Timeline (Week 3: March 26 to April 2):
* Analyze results and make hypotheses (3 hrs)
* Review project activites to verify correctness (1 hr)
* Summarize project and list some potential next steps (1 hr)

#### F

Planned Timeline (Week 4: April 2 to April 9):
* Write report (6 hrs)

___
### Data Understanding/Preparation

A subset of data from three datasets will be analyzed, cleaned, and linked in this notebook. These datasets include:
* San Francisco Ford GoBike Share (bike trips)
* NOAA Hourly Surface Data (weather)
* SFPD Incident Reports (crime)

All of these datasets will go towards predicting station-level demand prediction in bike-sharing systems. The aforementioned paper only uses bike trip and weather information to make predictions. However, they noted that oftentimes, crime rates are also correlated with bike share usage. Thus, due to the availability of incidents reports from the San Francisco Police Department, I will include an extra feature: the number of relevant crime incidents around a certain bike station per day. 

For the purposes of this version, I will only be working with data from January and February of 2018. The full data preparation will include data from January 2018 to February 2020. Following a similar partitioning as the paper, training will be done on data from January 2018 to July 2019, validation will be done on data from August to October 2019, and testing will be done on data from November 2019 to January 2020. 

**San Francisco Ford GoBike Share**
1. Data is split into different csv files, partitioned by month. Each csv file has the same schema. There are no empty entries and no malformed data. 
2. Stations are distinguished by names along with latitude/longitude coordinates. Encoding is not necessary as a separate model needs to be created and solved for each station. 

**NOAA Hourly Surface Data**
1. Data is split into different csv files, partitioned by year. Each csv file has the same schema. There are no empty entries under the relevant headings, but care needs to be taken to only include hourly reports of type FM-15. These hourly reports are also taken at minute 56 of each hour, so to simplify the data, four minutes are added to the timestamp so that each report timestamp refers to the end of the hour it has recorded data for. 
2. There are no categorical variables. 

**SFPD Incident Reports**
1. All data from the beginning of 2018 is included in one csv file. There are no empty entries under the relevant headings. However, there are many incidents which refer to non-relevant activity (e.g. hidden crimes, white-collar crimes, non-criminal incidents). Thus, all irrelevant entries need to be purged from the dataset so the number of incidents will accurately reflect what people are observing in the neighbourhood. 
2. Incidents are categorized by the type of crime/activity, but we are not concerned with that level of granularity in the data and only care about the number of relevant criminal incidents close to each station per day. Thus, there is no need to encode any of the incident categories. 

Initialize Spark...

In [34]:
# Add Java locations
import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_231.jdk/Contents/Home/"
os.environ["JRE_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_231.jdk/Contents/Home/"

# Initialize spark
import findspark
findspark.init("/usr/local/Cellar/apache-spark@2.3.2/2.3.2/libexec/")
import pyspark
spark = pyspark.sql.SparkSession.builder.appName("appName").getOrCreate()

Load trip data. 

In [35]:
data = []
for i in ['01', '02']:
    filepath = 'resources/2018' + i + '-fordgobike-tripdata.csv'
    month_data = spark.read.csv(filepath,
                               header=True,
                               inferSchema=True,
                               sep=',',
                               mode='DROPMALFORMED')
    data.append(month_data)
for i in range(1, len(data)):
    data[0] = data[0].union(data[i])
data[0].createOrReplaceTempView('tripData')
data[0].printSchema()

root
 |-- duration_sec: integer (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- bike_share_for_all_trip: string (nullable = true)



Load crime data and enforce schema. 

In [36]:
filepath = 'resources/sfpd_crime.csv'
crime_data = spark.read.csv(filepath,
                           header=True,
                           inferSchema=True,
                           sep=',',
                           mode='DROPMALFORMED')
crime_data.createOrReplaceTempView('temp')
crime_data = spark.sql("""
SELECT * FROM
    (
    SELECT TO_TIMESTAMP(`Incident Datetime`, "yyyy/MM/dd hh:mm") AS incident_timestamp,
    `Incident ID` AS incident_id,
    `Report Type Code` AS report_code,
    `Incident Code` AS incident_code,
    `Incident Category` AS incident_category,
    `Incident Subcategory` AS incident_subcategory,
    `Resolution` AS resolution,
    `Latitude` as lat,
    `Longitude` as long
    FROM temp
    WHERE `Latitude` IS NOT NULL
    AND `Incident Category` IS NOT NULL
    )
WHERE MONTH(incident_timestamp) <= 2
AND YEAR(incident_timestamp) = 2018
""")
crime_data.createOrReplaceTempView('crimeData')
crime_data.printSchema()

root
 |-- incident_timestamp: timestamp (nullable = true)
 |-- incident_id: integer (nullable = true)
 |-- report_code: string (nullable = true)
 |-- incident_code: integer (nullable = true)
 |-- incident_category: string (nullable = true)
 |-- incident_subcategory: string (nullable = true)
 |-- resolution: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)



Load weather data and enforce schema. 

In [37]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import split

filepath = 'resources/sf_weather.csv'
weather_data = spark.read.csv(filepath,
                              header=True,
                              inferSchema=True,
                              sep=',',
                              mode='DROPMALFORMED')
weather_data.createOrReplaceTempView('temp')
weather_data = spark.sql("""
SELECT * FROM
    (
    SELECT (DATE + INTERVAL '4' MINUTE) AS timestamp,
    LATITUDE AS lat,
    LONGITUDE AS long,
    WND AS wind_speed_rate,
    VIS AS visibility,
    TMP AS temperature,
    AA1 AS precipitation
    FROM temp
    WHERE SOURCE = 7 AND REPORT_TYPE = "FM-15"
    )
WHERE MONTH(timestamp) <= 2
""")
weather_data = weather_data.withColumn('wind_speed_rate', split('wind_speed_rate', '\,')[3])
weather_data = weather_data.withColumn('wind_speed_rate', weather_data['wind_speed_rate'].cast(IntegerType()))
weather_data = weather_data.withColumn('visibility', split('visibility', '\,')[0])
weather_data = weather_data.withColumn('visibility', weather_data['visibility'].cast(IntegerType()))
weather_data = weather_data.withColumn('temperature', split('temperature', '\,')[0])
weather_data = weather_data.withColumn('temperature', weather_data['temperature'].cast(IntegerType()))
weather_data = weather_data.withColumn('temperature', weather_data['temperature'] / 10)
weather_data = weather_data.withColumn('precipitation', split('precipitation', '\,')[1])
weather_data = weather_data.withColumn('precipitation', weather_data['precipitation'].cast(IntegerType()))
weather_data.createOrReplaceTempView('weatherData')
weather_data.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- wind_speed_rate: integer (nullable = true)
 |-- visibility: integer (nullable = true)
 |-- temperature: double (nullable = true)
 |-- precipitation: integer (nullable = true)



Aggregate trip data and replace table with new data. 

In [38]:
aggregate = spark.sql("""
SELECT
COALESCE(departure_count, 0) AS departure_count,
COALESCE(arrival_count, 0) AS arrival_count,
COALESCE(departure.hour, arrival.hour) AS hour,
COALESCE(departure.day, arrival.day) AS day,
COALESCE(departure.month, arrival.month) AS month,
COALESCE(departure.start_station_name, arrival.end_station_name) AS station_name,
COALESCE(departure.lat, arrival.lat) AS lat,
COALESCE(departure.long, arrival.long) AS long
FROM
    (SELECT COUNT(1) AS departure_count, hour, day, month, start_station_name, lat, long
    FROM
        (
        SELECT MONTH(start_time) AS month, DAY(start_time) AS day, HOUR(start_time) AS hour, start_station_name, start_station_latitude AS lat, start_station_longitude AS long
        FROM tripData
        )
    GROUP BY start_station_name, lat, long, hour, day, month
    ORDER BY month, day, hour, start_station_name) AS departure
FULL OUTER JOIN
    (SELECT COUNT(1) AS arrival_count, hour, day, month, end_station_name, lat, long
    FROM
        (
        SELECT MONTH(end_time) AS month, DAY(end_time) AS day, HOUR(end_time) AS hour, end_station_name, end_station_latitude AS lat, end_station_longitude AS long
        FROM tripData
        )
    GROUP BY month, day, hour, end_station_name, lat, long
    ORDER BY month, day, hour, end_station_name) AS arrival
ON
    departure.start_station_name = arrival.end_station_name AND
    departure.month = arrival.month AND
    departure.day = arrival.day AND
    departure.hour = arrival.hour
""")
aggregate.write.mode('overwrite').saveAsTable('aggregateTripData')

Analyze weather data. 

In [39]:
spark.sql("""
SELECT *
FROM
    (
    SELECT COUNT(1) AS num_hours, DAY(timestamp) AS day, MONTH(timestamp) AS month
    FROM weatherData
    GROUP BY day, month
    ORDER BY month, day
    )
WHERE num_hours < 24
""").show()
spark.sql("""
SELECT *
FROM weatherData
WHERE MONTH(timestamp) = 1
AND (DAY(timestamp) = 1 OR DAY(timestamp) = 31)
""").show()

+---------+---+-----+
|num_hours|day|month|
+---------+---+-----+
|       23|  1|    1|
|       23| 31|    1|
+---------+---+-----+

+-------------------+-------+---------+---------------+----------+-----------+-------------+
|          timestamp|    lat|     long|wind_speed_rate|visibility|temperature|precipitation|
+-------------------+-------+---------+---------------+----------+-----------+-------------+
|2018-01-01 01:00:00|37.6197|-122.3647|             15|     12875|       13.9|            0|
|2018-01-01 02:00:00|37.6197|-122.3647|             21|     16093|       13.3|            0|
|2018-01-01 03:00:00|37.6197|-122.3647|             21|     16093|       12.8|            0|
|2018-01-01 04:00:00|37.6197|-122.3647|              0|     16093|       12.2|            0|
|2018-01-01 05:00:00|37.6197|-122.3647|             15|     16093|       12.2|            0|
|2018-01-01 06:00:00|37.6197|-122.3647|              0|     16093|       12.2|            0|
|2018-01-01 07:00:00|37.6197|-

There is no entry for 00:00 on January 1, 2018 because the hourly data is in reference to the hour designated by its end time in *timestamp*, so we can just copy the data from 01:00 on January 1. But, it appears we are missing data for the hour 07:00 to 08:00 on January 31, 2018, so to compensate, we can pull data from the preceding and following entry to fill in that blank with an average. 

In [40]:
new_entry0 = spark.sql("""
SELECT '2018-01-01 00:00:00', lat, long, wind_speed_rate,
visibility, temperature, precipitation
FROM weatherData
WHERE timestamp = '2018-01-01 01:00:00'
""")
new_entry1 = spark.sql("""
SELECT '2018-01-31 08:00:00', AVG(lat), AVG(long), AVG(wind_speed_rate),
AVG(visibility), AVG(temperature), AVG(precipitation)
FROM weatherData
WHERE timestamp = '2018-01-31 07:00:00'
OR timestamp = '2018-01-31 09:00:00'
""")
weather_data = weather_data.union(new_entry0)
weather_data = weather_data.union(new_entry1)
weather_data = weather_data.orderBy(['timestamp'])
weather_data.createOrReplaceTempView('weatherData')

In [41]:
spark.sql("""
SELECT * FROM weatherData
WHERE DAY(timestamp) = 31
OR DAY(timestamp) = 1
""").show()

+-------------------+-------+---------+---------------+----------+-----------+-------------+
|          timestamp|    lat|     long|wind_speed_rate|visibility|temperature|precipitation|
+-------------------+-------+---------+---------------+----------+-----------+-------------+
|2018-01-01 00:00:00|37.6197|-122.3647|           15.0|   12875.0|       13.9|          0.0|
|2018-01-01 01:00:00|37.6197|-122.3647|           15.0|   12875.0|       13.9|          0.0|
|2018-01-01 02:00:00|37.6197|-122.3647|           21.0|   16093.0|       13.3|          0.0|
|2018-01-01 03:00:00|37.6197|-122.3647|           21.0|   16093.0|       12.8|          0.0|
|2018-01-01 04:00:00|37.6197|-122.3647|            0.0|   16093.0|       12.2|          0.0|
|2018-01-01 05:00:00|37.6197|-122.3647|           15.0|   16093.0|       12.2|          0.0|
|2018-01-01 06:00:00|37.6197|-122.3647|            0.0|   16093.0|       12.2|          0.0|
|2018-01-01 07:00:00|37.6197|-122.3647|           15.0|   16093.0|    

Aggregate clean weather data with trip data. 

In [42]:
aggregate_trip_weather = spark.sql("""
SELECT *
FROM
    (
    SELECT b.month, b.day, b.hour, station_name, lat, long, departure_count, arrival_count,
    wind_speed_rate, visibility, temperature, precipitation
    FROM
        (SELECT MONTH(timestamp) AS month,
        DAY(timestamp) AS day,
        HOUR(timestamp) AS hour,
        wind_speed_rate,
        visibility,
        temperature,
        precipitation
        FROM weatherData) AS a
    FULL OUTER JOIN
        aggregateTripData AS b
    ON a.month = b.month
    AND a.day = b.day
    AND a.hour = b.hour
    )
WHERE month IS NOT NULL
""")
aggregate_trip_weather.createOrReplaceTempView('aggregateTripWeatherData')

Make sure all stations are captured in start stations from trip data by ensuring the count of start and end stations are the same. This will allow us to just use the lat/long coordinates of the start stations to calculate which station each crime incident happens closest to. 

In [43]:
spark.sql("""
SELECT COUNT(*) as start_count
FROM
    (
    SELECT DISTINCT start_station_name AS name, start_station_latitude AS lat, start_station_longitude AS long
    FROM tripData
    )
""").show()
spark.sql("""
SELECT COUNT(*) as end_count
FROM
    (
    SELECT DISTINCT end_station_name AS name, end_station_latitude AS lat, end_station_longitude AS long
    FROM tripData
    )
""").show()

+-----------+
|start_count|
+-----------+
|        277|
+-----------+

+---------+
|end_count|
+---------+
|      277|
+---------+



There are several categories an incident can be placed in, but there most likely exists some categories that will not impact human behaviour within a specific neighbourhood, such as accidental fires, suicides, white-collar crimes, etc. Incidents relating to these categories should be removed such that they do not skew the data. 

In [44]:
spark.sql("""
SELECT incident_category FROM crimeData
GROUP BY incident_category
""").show(50, False)

+------------------------------------------+
|incident_category                         |
+------------------------------------------+
|Vehicle Misplaced                         |
|Suspicious                                |
|Forgery And Counterfeiting                |
|Sex Offense                               |
|Family Offense                            |
|Fire Report                               |
|Assault                                   |
|Recovered Vehicle                         |
|Drug Violation                            |
|Robbery                                   |
|Motor Vehicle Theft?                      |
|Embezzlement                              |
|Vehicle Impounded                         |
|Missing Person                            |
|Rape                                      |
|Lost Property                             |
|Arson                                     |
|Fraud                                     |
|Homicide                                  |
|Drug Offe

There appear to be several categories that will not impact visible crime within an area. These categories are:
* Vehicle Misplaced
* Forgery and Counterfeiting
* Fire Report
* Recovered Vehicle
* Motor Vehicle Theft? (this appears to be a mistake, aggregate with Motor Vehicle Theft)
* Vehicle Impounded
* Embezzlement
* Lost Property
* Suicide
* Other
* Gambling
* Warrant
* Courtesy Report
* Missing Person
* Miscellaneous Investigation
* Non-Criminal
* Civil Sidewalks
* Case Closure
* Other Miscellaneous
* Other Offenses

Incidents relating to these categories should be removed. 

In [45]:
import pyspark.sql.functions as F

crime_data = spark.sql("""
SELECT * FROM crimeData
WHERE incident_category != "Vehicle Misplaced"
AND incident_category != "Forgery And Counterfeiting"
AND incident_category != "Fire Report"
AND incident_category != "Recovered Vehicle"
AND incident_category != "Vehicle Impounded"
AND incident_category != "Embezzlement"
AND incident_category != "Lost Property"
AND incident_category != "Suicide"
AND incident_category != "Other"
AND incident_category != "Gambling"
AND incident_category != "Warrant"
AND incident_category != "Courtesy Report"
AND incident_category != "Miscellaneous Investigation"
AND incident_category != "Non-Criminal"
AND incident_category != "Civil Sidewalks"
AND incident_category != "Case Closure"
AND incident_category != "Other Miscellaneous"
AND incident_category != "Missing Person"
AND incident_category != "Other Offenses"
""")
crime_data = crime_data.withColumn('incident_category', F.when(crime_data['incident_category'] == 'Motor Vehicle Theft?', 'Motor Vehicle Theft').otherwise(crime_data['incident_category']))
crime_data.createOrReplaceTempView('crimeData')
spark.sql("""
SELECT incident_category FROM crimeData
GROUP BY incident_category
""").show(50, False)

+------------------------------------------+
|incident_category                         |
+------------------------------------------+
|Suspicious                                |
|Sex Offense                               |
|Family Offense                            |
|Assault                                   |
|Drug Violation                            |
|Robbery                                   |
|Rape                                      |
|Arson                                     |
|Fraud                                     |
|Homicide                                  |
|Drug Offense                              |
|Burglary                                  |
|Human Trafficking (A), Commercial Sex Acts|
|Traffic Violation Arrest                  |
|Traffic Collision                         |
|Weapons Carrying Etc                      |
|Weapons Offense                           |
|Prostitution                              |
|Suspicious Occ                            |
|Larceny T

Assign each crime incident to its nearest bike station using lat/long coordinates. 

In [46]:
bike_stations = spark.sql("""
SELECT DISTINCT start_station_name AS name, start_station_latitude AS lat, start_station_longitude AS long
FROM tripData
""")
bike_stations.createOrReplaceTempView('bikeStations')

In [47]:
crime_to_station = spark.sql("""
SELECT *
FROM
    (
    SELECT name, incident_timestamp, incident_id, distance, DENSE_RANK() OVER (PARTITION BY incident_timestamp, incident_id ORDER BY distance) AS rank
    FROM
        (
        SELECT a.name, b.incident_timestamp, b.incident_id, 
        acos(sin(radians(a.lat)) * sin(radians(b.lat)) +
                cos(radians(a.lat)) * cos(radians(b.lat)) *
                cos(radians(a.long - b.long))) * 6372.8 AS distance
        FROM
            bikeStations AS a
        CROSS JOIN
            crimeData AS b
        )
    )
WHERE rank = 1
""")
crime_to_station.write.mode('overwrite').saveAsTable('crimeToStation')

Make sure each crime incident has been assigned a station by comparing the size of the two datasets. 

In [48]:
print(crime_to_station.count(), crime_data.count())

16569 16569


Aggregate crime incidents per day and join with aggregated trip/weather data. It is better to use the previous day's crime rate than the daily crime incidents as a feature because the daily rate is unknown during that day. If there is no entry in crime incident table, then there were 0 incidents on that day for that station, so all the null values in *num_incidents* can be replaced with 0. 

In [49]:
aggregate_tripWeatherCrime = spark.sql("""
SELECT *
FROM
    (
    SELECT month, day, hour, station_name, lat, long, departure_count, arrival_count, wind_speed_rate, visibility, temperature, precipitation, num_incidents
    FROM
        (SELECT COUNT(1) AS num_incidents, name, DATE(incident_timestamp) AS date
        FROM crimeToStation
        GROUP BY date, name) AS a
    FULL OUTER JOIN
        aggregateTripWeatherData AS b
    ON CAST(CONCAT("2018-", month, "-", day) AS date) = DATE_ADD(date, 1)
    AND a.name = b.station_name
    )
WHERE month IS NOT NULL
""")
aggregate_tripWeatherCrime = aggregate_tripWeatherCrime.withColumn('num_incidents', F.when(aggregate_tripWeatherCrime['num_incidents'].isNull(), 0).otherwise(aggregate_tripWeatherCrime['num_incidents']))
aggregate_tripWeatherCrime.coalesce(1).write.format('parquet').mode('overwrite').save('aggregateTripWeatherCrimeData.parquet')

Now the table **aggregateTripWeatherCrimeData** is cleaned and structured. All relevant information have been included and linked. 

___
### Bibliography

\[1\] P. Hulot, D. Aloise, and S.D. Jena, "Towards Station-Level Demand Prediction for Effective Rebalancing in Bike-Sharing Systems," In KDD'18: Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discover \& Data Mining, 2018, pp. 378-386. 